In [2]:
from pathlib import Path
import os

# Establece la raíz del proyecto manualmente
project_root = Path("F:/JCMDataCenter/Cursos/Evolve Academy/Data Scientist IA/Futpeak") # sobremesa
#project_root = Path("C:/Users/juanm/Desktop/FUTPEAK/Futpeak") # portátil

# Cambia el directorio de trabajo actual a esa raíz
os.chdir(project_root)

print("📁 Directorio de trabajo actual:", Path.cwd())


📁 Directorio de trabajo actual: F:\JCMDataCenter\Cursos\Evolve Academy\Data Scientist IA\Futpeak


In [3]:
import subprocess

# Ejecuta pip freeze y filtra las líneas inválidas
output = subprocess.check_output(["pip", "freeze"], text=True)
clean_lines = [line for line in output.splitlines() if "@ file://" not in line and "file:///" not in line]

# Escribe el requirements limpio
with open("requirements.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(clean_lines))

print("✅ requirements.txt limpio generado.")


✅ requirements.txt limpio generado.


In [10]:
import pandas as pd
import os

# Cargar los archivos
metadata_df = pd.read_csv('data/processed/male_only_metadata.csv')
matchlogs_df = pd.read_csv('data/processed/cleaned_matchlogs.csv')

# Obtener IDs válidos desde metadata
male_ids = set(metadata_df['Player_ID'])

# Filtrar matchlogs por esos jugadores
filtered_matchlogs_df = matchlogs_df[matchlogs_df['Player_ID'].isin(male_ids)]

# Eliminar columnas duplicadas del matchlogs que vendrán del metadata
filtered_matchlogs_df = filtered_matchlogs_df.drop(columns=[col for col in ['Player_name', 'Position'] if col in filtered_matchlogs_df.columns])

# Merge con metadata
merged_df = filtered_matchlogs_df.merge(
    metadata_df[['Player_ID', 'Player_name', 'Full_name', 'Birth_date', 'Age', 'Position', 'Footed', 'Birth_place', 'Nationality', 'Club']],
    on='Player_ID',
    how='left'
)

# Reordenar columnas
initial_cols = ['Player_name', 'Player_ID']
merged_df = merged_df[[col for col in initial_cols if col in merged_df.columns] + [col for col in merged_df.columns if col not in initial_cols]]

In [11]:
print(f"Filas antes del merge: {len(filtered_matchlogs_df)}")
print(f"Filas después del merge: {len(merged_df)}")
print("Columnas nuevas añadidas:", set(merged_df.columns) - set(filtered_matchlogs_df.columns))


Filas antes del merge: 421097
Filas después del merge: 421097
Columnas nuevas añadidas: {'Birth_date', 'Position', 'Player_name', 'Nationality', 'Club', 'Footed', 'Full_name', 'Birth_place', 'Age'}


In [ ]:
# Jugadores únicos en metadata
print("Metadata jugadores únicos:", metadata_df['Player_ID'].nunique())

# Jugadores únicos en matchlogs
print("Matchlogs jugadores únicos:", matchlogs_df['Player_ID'].nunique())

# Jugadores únicos en merged_df (es decir, con partidos y metadata masculina)
print("Merged jugadores únicos:", merged_df['Player_ID'].nunique())

# Los resultados nos indican que no tenemos paridos de todos los jugadores en metadata y que tenemos matchlogs de chicas también, por eso obtenemos 2282 chicos con partidos en matchlogs e info en metadata.


Metadata jugadores únicos: 3344
Matchlogs jugadores únicos: 2809
Merged jugadores únicos: 2282


In [ ]:
# Obtener la cantidad de jugadores únicos que tienen partidos en matchlogs
unique_players = merged_df['Player_ID'].nunique()
print(f"🎯 Jugadores únicos: {unique_players}")

# Listar todas las columnas del DataFrame
print("🧾 Columnas del DataFrame:")
for col in merged_df.columns:
    print(f"- {col}")


🎯 Jugadores únicos: 2282
🧾 Columnas del DataFrame:
- Player_name
- Player_ID
- Seasons
- Date
- Day
- Competition
- Round
- Home_Away
- Result
- Player_team
- Rival_team
- Start
- Minutes
- Goals
- Assists
- Penalty_kick
- Penalty_kick_att
- Shots
- Shots_on_target
- Yellow_cards
- Red_cards
- Fouls_committed
- Fouls_drawn
- Offsides
- Crosses
- Tackles_won
- Interceptions
- Own_goals
- Penaltys_won
- Penaltys_conceded
- Touches
- Tackles
- Blocks
- xG
- non_penalty_xG
- x_assisted_G
- Shot_creating_actions
- Goal_creating_actions
- Passes_completed
- Passes_att
- Percent_passes
- Progressive_passes
- Feet_control
- Progressive_control
- Dribling_suc
- Full_name
- Birth_date
- Age
- Position
- Footed
- Birth_place
- Nationality
- Club


In [16]:
# Calcular top 10 de jugadores con más temporadas distintas
top_seasons = (
    merged_df.groupby(['Player_ID', 'Player_name'])['Seasons']
    .nunique()
    .sort_values(ascending=False)
    .head(10)
    .reset_index(name='Num_Seasons')
)

# Mostrar resultado
print(top_seasons)


  Player_ID        Player_name  Num_Seasons
0  d5ff6084   Marcos Guilherme           15
1  d4814e00     Fernando Zuqui           15
2  b14f755d       Ivan Marcone           15
3  cf5c3ef1  Carlos Izquierdoz           15
4  1edc043c   Federico Carrizo           15
5  b4519dd4     Nicolas Blandi           15
6  4dcde17a              Cesar           14
7  476d6fbe      Carlos Auzqui           14
8  374d5158    Gonzalo Montiel           14
9  853b7c48  Agustin Marchesin           14


In [18]:
# Top 10 jugadores con más partidos (conteo de filas por jugador)
top_matches = (
    merged_df.groupby(['Player_ID', 'Player_name'])
    .size()
    .sort_values(ascending=False)
    .head(10)
    .reset_index(name='Num_Matches')
)

# Top 10 jugadores con más minutos jugados
top_minutes = (
    merged_df.groupby(['Player_ID', 'Player_name'])['Minutes']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index(name='Total_Minutes')
)

# Mostrar resultados
print("🏟️ Top 10 jugadores con más partidos:")
print(top_matches)

print("\n⏱️ Top 10 jugadores con más minutos jugados:")
print(top_minutes)


🏟️ Top 10 jugadores con más partidos:
  Player_ID       Player_name  Num_Matches
0  a1d5bd30   Marcus Rashford          505
1  42fd9c7f     Kylian Mbappe          504
2  162efffd   Rodrigo De Paul          497
3  01eb744d      Angel Correa          491
4  7a2e46a8           Alisson          478
5  f7036e1c  Lautaro Martinez          457
6  81be82e9          Weverton          455
7  b3e42951     Franco Armani          451
8  81560b83   Jordan Turnbull          449
9  b66315ae     Gabriel Jesus          446

⏱️ Top 10 jugadores con más minutos jugados:
  Player_ID      Player_name  Total_Minutes
0  7a2e46a8          Alisson        42977.0
1  81be82e9         Weverton        40924.0
2  b3e42951    Franco Armani        40713.0
3  81560b83  Jordan Turnbull        39200.0
4  42fd9c7f    Kylian Mbappe        38581.0
5  d8f11068    Nahuel Guzman        38547.0
6  fba79ac8      Tiago Volpi        37858.0
7  aa7dfc40      Jake Cooper        37771.0
8  9fc6318b    Brad Halliday        35646.0
9  

In [19]:
# Paso 1: obtener jugadores únicos con su país
unique_players = merged_df[['Player_ID', 'Nationality']].drop_duplicates()

# Paso 2: contar jugadores por país
top_nationalities = unique_players['Nationality'].value_counts().head(10)
print(top_nationalities)



Nationality
England          516
Brazil           499
Argentina        447
Spain            355
France           296
Germany          130
Côte d'Ivoire      3
Andorra            3
Monaco             2
United States      2
Name: count, dtype: int64


In [21]:
merged_df['Goals'] = pd.to_numeric(merged_df['Goals'], errors='coerce')
top_goals = merged_df.groupby(['Player_ID', 'Player_name'])['Goals'].sum().sort_values(ascending=False).head(10)
print(top_goals)


Player_ID  Player_name            
42fd9c7f   Kylian Mbappe              367.0
f7036e1c   Lautaro Martinez           209.0
bb410065   Marie Antoinette Katoto    178.0
e7ef2c4e   Paul Mullin                177.0
9ccf4e14   German Cano                176.0
6d764149   Michael Cheek              167.0
b66315ae   Gabriel Jesus              160.0
a1d5bd30   Marcus Rashford            159.0
5c4f4e43   Talisca                    155.0
1b9399bc   Bernardo Cuesta            153.0
Name: Goals, dtype: float64


In [23]:
# 🅰️ Top asistentes

top_assists = (
    merged_df.groupby(['Player_ID', 'Player_name'])['Assists']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index(name='Total_Assists')
)
top_assists.head()

,Player_ID,Player_name,Total_Assists
0,42fd9c7f,Kylian Mbappe,139.0
1,cd1acf9d,Trent Alexander Arnold,95.0
2,b19db005,Ousmane Dembele,95.0
3,dbf053da,Jadon Sancho,83.0
4,3423f250,Raphinha,82.0


In [24]:
# ⚽ Goles + Asistencias combinadas

top_goal_contributions = (
    merged_df.groupby(['Player_ID', 'Player_name'])[['Goals', 'Assists']]
    .sum()
    .assign(GA=lambda df: df['Goals'] + df['Assists'])
    .sort_values(by='GA', ascending=False)
    .head(10)
    .reset_index()
)
top_goal_contributions.head()

,Player_ID,Player_name,Goals,Assists,GA
0,42fd9c7f,Kylian Mbappe,367.0,139.0,506.0
1,f7036e1c,Lautaro Martinez,209.0,44.0,253.0
2,b66315ae,Gabriel Jesus,160.0,78.0,238.0
3,a1d5bd30,Marcus Rashford,159.0,71.0,230.0
4,bb410065,Marie Antoinette Katoto,178.0,45.0,223.0


In [25]:
# 📈 Promedio de goles por partido

avg_goals_per_game = (
    merged_df.groupby(['Player_ID', 'Player_name'])['Goals']
    .mean()
    .sort_values(ascending=False)
    .head(10)
    .reset_index(name='Avg_Goals_per_Game')
)
avg_goals_per_game.head()

,Player_ID,Player_name,Avg_Goals_per_Game
0,bb410065,Marie Antoinette Katoto,0.843602
1,42fd9c7f,Kylian Mbappe,0.728175
2,5c20d17f,Dominic Mchale,0.666667
3,823630ad,Anthony Vermet,0.545455
4,5c4f4e43,Talisca,0.530822


In [26]:
# 🟨 Tarjetas amarillas

top_yellows = (
    merged_df.groupby(['Player_ID', 'Player_name'])['Yellow_cards']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index(name='Total_Yellows')
)
top_yellows.head()

,Player_ID,Player_name,Total_Yellows
0,74e0a984,Damian Musto,126.0
1,5b37f2f8,Walter Kannemann,123.0
2,b1805c80,Alexander Barboza,122.0
3,c3721674,Carlos Javier Beltran,111.0
4,ea8d470b,Agustin Farias,110.0


In [27]:
# 🟥 Tarjetas rojas

top_reds = (
    merged_df.groupby(['Player_ID', 'Player_name'])['Red_cards']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index(name='Total_Reds')
)
top_reds.head()

,Player_ID,Player_name,Total_Reds
0,fb29d05d,Emanuel Britez,15.0
1,b3aed40d,Raul Silva,12.0
2,c3721674,Carlos Javier Beltran,12.0
3,b1805c80,Alexander Barboza,11.0
4,2c20f2fb,Cristian Lema,10.0


In [28]:
# 🏟️ Jugadores por club

players_per_club = (
    merged_df.groupby('Club')['Player_ID']
    .nunique()
    .sort_values(ascending=False)
    .head(10)
    .reset_index(name='Unique_Players')
)
players_per_club.head()

,Club,Unique_Players
0,Fortaleza,16
1,River Plate,15
2,Flamengo,13
3,United Kingdom,12
4,Santos,12


In [ ]:
# 📌 Distribución por posición CORREGIR

position_distribution = (
    merged_df.groupby('Position')
    .nunique()
    .value_counts()
    .reset_index(name='Count')
    .rename(columns={'index': 'Position'})
)
position_distribution.head()

In [ ]:
# 🧭 Top nacionalidades CORREGIR

top_nationalities = (
    merged_df.groupby('Nationality')['Player_ID']
    .nunique()
    .sort_values(ascending=False)
    .head(10)
    .reset_index(name='Num_Players')
)
top_nationalities.head()

In [ ]:
# 📊 Edad promedio por posición CORREGIR

avg_age_position = (
    merged_df.groupby('Position')['Age']
    .mean()
    .sort_values(ascending=False)
    .reset_index(name='Avg_Age')
)
avg_age_position.head()

In [33]:
# 🦶 Distribución por pie dominante

foot_distribution = (
    merged_df['Footed']
    .value_counts()
    .reset_index(name='Count')
    .rename(columns={'index': 'Footed'})
)
foot_distribution.head()

,Footed,Count
0,Right,255388
1,Left,79259
